### import

In [64]:
import os, re
import pandas as pd
import numpy as np
from os.path import join
from datetime import datetime, timedelta
import warnings
warnings.simplefilter("ignore")

# 나우리 타입 반영
nauri_dtype = {"채무자키":str, "계좌키":str, '보증인키':str, '신용회복키' : str, "계좌번호":str, "심의차수":str, "변제금수취계좌":str}

# 진행구분열 작성-------------------------------
def 신용회복_진행구분작성(df, 기준데이터기준적용=False) :
    
    계좌진행상태 = df.계좌진행상태내용 # 신청인보다 계좌먼저처리해야 함.
    신청인진행 = df.신청인진행상태
    계좌번호 = df.계좌번호

    conds = [
        (계좌진행상태 == "채무조정제외"), # 제외처리 먼저
        (계좌진행상태.str.contains(r"완제|상환") | 계좌번호.str.contains("완제")), # 완제일없는 완제 : 개별상환, 완제(중도), 상환완료(계좌별):630 | 계좌번호 : 9999999999채무 완제-접수당시완제
        # 이제 계좌 : 포함과 기타(채권신고전)만 남음
        (신청인진행.str.contains("완제")), # 완제일있는 완제 : 중도완제, 만기완제
        (신청인진행 == '정상이행'),
        (계좌번호 == "채권신고전"), # == 계좌진행 : 기타 
        (신청인진행 == "심사반송"), # 심사반송은 날짜 없음
        (신청인진행.str.contains(r"실효|포기")), # 중도실효, 합의서체결포기
    ]
    if 기준데이터기준적용 : 
        values = ["채무조정제외","완제", "완제", "확정", "채권신고전", "실효", "실효"]
    else : 
        values = ["채무조정제외","완제_날짜없음", "완제_날짜있음", "확정", "채권신고전", "실효_날짜없음", "실효_날짜있음"]
    default_value = "미확정"

    df["진행구분"] = np.select(conds, values, default_value)
    return df


In [96]:
# 계좌별 진행상황 파일 읽기
path_base = r"D:\3.자산\신용회복\신용회복 전체데이터\계좌별 진행상황"
########################################################
지난주 = "240131"
이번주 = "240202"
company = "통합"  # 통합 / 대성
########################################################
파일명 = "_계좌별 진행상황 조회_"+ company + "_업로드결과"

# 파일 읽기
이전파일_ori = pd.read_excel(join(path_base, 지난주 + 파일명 +".xlsx"), dtype=nauri_dtype).fillna("")
현재파일_ori = pd.read_excel(join(path_base, 이번주 + 파일명 +".xlsx"), dtype=nauri_dtype).fillna("")

In [65]:
이전파일 = 이전파일_ori.copy()
현재파일 = 현재파일_ori.copy()

# 진행구분 작성하고
# 키값없는 거는 모두 예전에 실효완제된 건이므로 지금은 신경쓰지 않아도 됨
이전파일 = 신용회복_진행구분작성(이전파일)
현재파일 = 신용회복_진행구분작성(현재파일)

# 신용회복키 없는 거 삭제
이전파일 = 이전파일[이전파일["신용회복키"]!=""]
현재파일 = 현재파일[현재파일["신용회복키"]!=""]

# 확인
print(현재파일.신용회복키.duplicated().value_counts())

신용회복키
False    18032
Name: count, dtype: int64


In [66]:
# 진행구분 달라진 거 찾기
cols = ["채무자키", "계좌키","계좌번호", "보증인키","신용회복키", "신청인명", "신청인주민번호", "거래일", "진행구분", "신청인진행상태", "계좌진행상태내용","실효/완제/합의서포기일자", "채무구분"]
merged = 현재파일[cols].merge(이전파일[["신용회복키", "진행구분", "신청인진행상태", "계좌진행상태내용", "실효/완제/합의서포기일자"]], on="신용회복키", how='left', suffixes=['_현재','_이전'])
merged = merged.fillna('') #### 이래야 str 메소드 쓸때 오류 안남. str메소드 쓸데 na처리 해주든가.

In [87]:
# 다운받은날(월요일)에 실효완제된 것도 포함한다.
완제 = merged.query('진행구분_현재.str.contains("완제") and ~진행구분_이전.str.contains("완제")')
실효 = merged.query('진행구분_현재.str.contains("실효") and ~진행구분_이전.str.contains("실효")')

# 날짜 있어야 하는 것 중, 새로 실효 및 완제했는데(따라서 완제, 실효df만든후에 실행해야함) 날짜값이 없다면 다운받은날이 실효,완제일이다.
완제.loc[(완제['진행구분_현재'].str.contains("날짜있음")) & (완제["실효/완제/합의서포기일자_현재"]==""),"실효/완제/합의서포기일자_현재"] = 완제.거래일.values[0] # 거래일 = 파일 다운로드일(사실 다음날일 수도 있긴 함....)
실효.loc[(실효['진행구분_현재'].str.contains("날짜있음")) & (실효["실효/완제/합의서포기일자_현재"]==""),"실효/완제/합의서포기일자_현재"] = 완제.거래일.values[0] # 거래일은 모두 동일함

print(len(완제), len(실효))

13 3


### 출력

In [48]:
출력열 = ["채무자키","계좌키","계좌번호","보증인키","실효/완제/합의서포기일자_현재","신청인명","신청인주분번호","채무구분"]
출력dir = r"D:\3.자산\신용회복\주간 완제 실효 신탁해지"

시작날짜 = datetime.strptime(지난주, "%y%m%d") + timedelta(days=1) # 하루보태고
시작날짜 = datetime.strftime(시작날짜, "%y%m%d") # 문자열로

com = "솔림" if company =="통합" else "대성"
filename_write =  시작날짜 +"~"+ 이번주 + "_" + com + "_주간 신용회복 완제실효"
fullpath_result = join(os.path.dirname(출력dir), filename_write +".xlsx")

# 엑셀 파일로 저장
if not os.path.exists(fullpath_result):
    param = {'mode' : 'w'}
else : 
    param = {'mode' : 'a', 'if_sheet_exists' : 'replace'}

with pd.ExcelWriter(fullpath_result, engine='openpyxl', **param) as writer :
    완제[출력열].to_excel(writer, sheet_name='완제', index=False)
    실효[출력열].to_excel(writer, sheet_name='실효', index=False)

,채무자키,계좌키,계좌번호,보증인키,신용회복키,신청인명,신청인주민번호,거래일,진행구분_현재,신청인진행상태_현재,계좌진행상태내용_현재,실효/완제/합의서포기일자_현재,진행구분_이전,신청인진행상태_이전,계좌진행상태내용_이전,실효/완제/합의서포기일자_이전
5367,20453119,200976827,L42005200650001,,21063067,이상호,760815-1696111,2024-02-04,완제,심사결재,개별상환,,실효,정상이행,채무조정제외,
7222,20479930,201001187,5137920026306252,,21160505,김경연,711212-2036718,2024-02-04,완제,정상이행,개별상환,,확정,정상이행,채무조정포함,
11758,20459910,200979960,1501000413,,21055272,유재필,550325-1255416,2024-02-04,완제,만기완제,채무조정포함,2024-02-02,확정,정상이행,채무조정포함,
11759,20459912,200979962,1501000413_2,,21055274,유재필,550325-1255416,2024-02-04,완제,만기완제,채무조정포함,2024-02-02,확정,정상이행,채무조정포함,
11760,20459911,200979961,1501000413_1,,21055273,유재필,550325-1255416,2024-02-04,완제,만기완제,채무조정포함,2024-02-02,확정,정상이행,채무조정포함,
11905,20465270,200985320,1601000609,,21068499,고규범,900105-1953912,2024-02-04,완제,만기완제,채무조정포함,2024-02-01,확정,정상이행,채무조정포함,
11906,20465271,200985321,1601000934,,21068500,고규범,900105-1953912,2024-02-04,완제,만기완제,채무조정포함,2024-02-01,확정,정상이행,채무조정포함,
12945,20460938,200980988,8210000160_188155,,21058570,김상숙,670828-2637116,2024-02-04,완제,중도완제,채무조정포함,2024-02-02,확정,정상이행,채무조정포함,
13340,20464823,200984873,8106001550_01,,21067713,이연주,870125-2199331,2024-02-04,완제,중도완제,채무조정포함,2024-02-01,확정,정상이행,채무조정포함,
13583,20460125,200980175,55198760839291,,21056149,배상한,600118-1347913,2024-02-04,완제,중도완제,채무조정포함,,확정,정상이행,채무조정포함,
